In [35]:
#importing required dependencies
import pandas as pd
import os
from src.utils import download_image

Initially we need to do data preprocessing

In [36]:
DATASET_FOLDER = 'dataset/'

# initially getting only 100 rows of train data
train =  pd.read_csv(os.path.join(DATASET_FOLDER,'sample_train.csv'))
print(train)
train.apply( lambda row : download_image(row['image_link'],'images'), axis=1)

                                            image_link  group_id  entity_name  \
0    https://m.media-amazon.com/images/I/61I9XdN6OF...    748919  item_weight   
1    https://m.media-amazon.com/images/I/71gSRbyXmo...    916768  item_volume   
2    https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516  item_weight   
3    https://m.media-amazon.com/images/I/612mrlqiI4...    459516  item_weight   
4    https://m.media-amazon.com/images/I/617Tl40LOX...    731432  item_weight   
..                                                 ...       ...          ...   
494  https://m.media-amazon.com/images/I/518L45yHsf...    810266  item_weight   
495  https://m.media-amazon.com/images/I/7179JK26-1...    893692  item_weight   
496  https://m.media-amazon.com/images/I/719skH5g3D...    299791  item_weight   
497  https://m.media-amazon.com/images/I/81L5nXqWeL...    731432  item_weight   
498  https://m.media-amazon.com/images/I/61m-jQu+Za...    558832  item_weight   

       entity_value  
0    

0      None
1      None
2      None
3      None
4      None
       ... 
494    None
495    None
496    None
497    None
498    None
Length: 499, dtype: object

In [37]:
def preprocess_image(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path)
    
    # Resize the image to target size
    resized_img = cv2.resize(img, TARGET_SIZE)
    
    # Normalize the pixel values (convert to float and divide by 255)
    normalized_img = resized_img.astype(np.float32) / 255.0
    
    return normalized_img


In [38]:
import cv2
import os
import numpy as np

# Directory where all your downloaded images are stored
image_folder = "images/"
processed_folder = "processed_images/"  # Folder to save the preprocessed images

# Ensure the folder exists
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

# Target image size (height, width) for resizing
TARGET_SIZE = (224, 224)

# Function to preprocess the image (resize and normalize)
# Loop over all images in the directory
for image_file in os.listdir(image_folder):
    if image_file.endswith(".jpg"):  # Only process .jpg files
        # Get full path of the image
        image_path = os.path.join(image_folder, image_file)
        
        # Preprocess the image
        processed_image = preprocess_image(image_path)
        
        # Save the preprocessed image (optional if you want to store them)
        save_path = os.path.join(processed_folder, image_file)
        # Convert back to 0-255 range to save as an image
        save_image = (processed_image * 255).astype(np.uint8)
        cv2.imwrite(save_path, save_image)
        
        print(f"Processed and saved: {image_file}")

print("All images have been preprocessed!")


Processed and saved: 313aG8L3-rL.jpg
Processed and saved: 31aalwb8cdL.jpg
Processed and saved: 31EvJszFVfL.jpg
Processed and saved: 31o8WywVeTL.jpg
Processed and saved: 41-GUSWgzWL.jpg
Processed and saved: 410gyMVw-UL.jpg
Processed and saved: 413FQB0ZMLL.jpg
Processed and saved: 4147dGptveL.jpg
Processed and saved: 414maJ3RUML.jpg
Processed and saved: 41CKvM8JV7L.jpg
Processed and saved: 41egkCDTTML.jpg
Processed and saved: 41EjbFu-+yL.jpg
Processed and saved: 41fX-OWziML.jpg
Processed and saved: 41fYovCueJL.jpg
Processed and saved: 41J6cTfYEJL.jpg
Processed and saved: 41jz25vrqpL.jpg
Processed and saved: 41Kn+YOyPjL.jpg
Processed and saved: 41njJJg-ANL.jpg
Processed and saved: 41o+Ah16aOL.jpg
Processed and saved: 41UtmzUICML.jpg
Processed and saved: 41wv12SiMUS.jpg
Processed and saved: 41wvffSxB4L.jpg
Processed and saved: 41yG6r50SGL.jpg
Processed and saved: 41zNB8ZshmL.jpg
Processed and saved: 41ZqgLmAOpL.jpg
Processed and saved: 51+N63vjwWL.jpg
Processed and saved: 51-GFfHUt5L.jpg
P

In [39]:
processed_images = []

for image_file in os.listdir(image_folder):
    if image_file.endswith(".jpg"):
        image_path = os.path.join(image_folder, image_file)
        processed_image = preprocess_image(image_path)
        processed_images.append(processed_image)

# processed_images now contains all the preprocessed images as NumPy arrays


In [40]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Initialize the Label Encoder
le = LabelEncoder()

# Apply the label encoder to the 'entity_name' column
train['entity_name_encoded'] = le.fit_transform(train['entity_name'])

# Display the first few rows to check the encoding
print(train[['entity_name', 'entity_name_encoded']].head())


   entity_name  entity_name_encoded
0  item_weight                    1
1  item_volume                    0
2  item_weight                    1
3  item_weight                    1
4  item_weight                    1


In [41]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define input image shape (224x224x3 for color images)
input_shape = (224, 224, 3)

# Load pre-trained ResNet50 model (without the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model (do not train its layers)
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)  # Flatten the output of the convolutional layers
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(128, activation='relu')(x)

# Output layer (Adjust the number of units to match the number of classes in entity_value)
# Assuming entity_value is a numeric value, we use regression with 1 output
output = Dense(1, activation='linear')(x)

# Define the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 75,034,241 (286.23 MB)

 Trainable params: 51,446,529 (196.25 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [44]:
X = np.array(images)

# Extract the encoded entity values from the data (as target labels)
y = train['entity_value'].values
print(len(X))
print(len(y))

NameError: name 'images' is not defined

In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load preprocessed images from the previous step
# Assuming you stored them as NumPy arrays in a list (processed_images)

# Convert list to a NumPy array for easy manipulation
X = np.array(processed_images)

# Extract the encoded entity values from the data (as target labels)
t=train.groupby('image_link')
y = train['entity_value'].values  # Assuming 'entity_value' contains numeric labels

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)



ValueError: Found input variables with inconsistent numbers of samples: [493, 499]

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

# Save the model after training
model.save('entity_extraction_model.h5')



In [ ]:
# Evaluate the model on validation set
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Mean Absolute Error: {val_mae}")
